In [2]:
import numpy as np
from tqdm.notebook import tqdm
import time

In [3]:
global register_a
global register_b
global register_c

global ip

def read_combo(operand):
    global register_a
    global register_b
    global register_c
    if 0 <= operand <= 3: return operand
    if operand == 4: return register_a
    if operand == 5: return register_b
    if operand == 6: return register_c
    if operand == 7: raise ValueError("opcode 7 reserver")
    raise ValueError("opcode invalid")

def adv(operand):
    global register_a
    register_a = int(register_a / (2**read_combo(operand)))
    return ""

def bxl(operand):
    global register_b
    register_b = register_b ^ operand
    return ""
    
def bst(operand):
    global register_b
    register_b = read_combo(operand) % 8
    return ""

def jnz(operand):
    global ip
    if register_a != 0: ip = operand - 2
    return ""
    
def bxc(operand):
    global register_b
    register_b = register_b ^ register_c
    return ""
    
def out(operand):
    return str(read_combo(operand) % 8) + ","
    
def bdv(operand):
    global register_b
    register_b = int(register_a / (2**read_combo(operand)))
    return ""
    
def cdv(operand):
    global register_c
    register_c = int(register_a / (2**read_combo(operand)))
    return ""


opcode = {
    0: adv,
    1: bxl,
    2: bst,
    3: jnz,
    4: bxc,
    5: out,
    6: bdv,
    7: cdv
          }

In [4]:
def run_program(program, reg_a, reg_b, reg_c):
    global register_a
    global register_b
    global register_c
    global ip
    
    register_a = reg_a
    register_b = reg_b
    register_c = reg_c
    
    ip = 0
    
    std_out = ""
    
    repeated_state = {}
    
    while(0<= ip < len(program) - 1):
        state = (ip, register_a, register_b, register_c)
        if repeated_state.get(state, 0): break
        repeated_state[state] = 1

        std_out += opcode[program[ip]](program[ip + 1])
        ip += 2
        
        if len(std_out) > 30: break
        
        
    return std_out[:-1]


In [25]:
with open("d17_easy_input") as f:
    reg_a = f.readline()[:-1]
    reg_a = int(reg_a.split(":")[-1])
    
    reg_b = f.readline()[:-1]
    reg_b = int(reg_b.split(":")[-1])
    
    reg_c = f.readline()[:-1]
    reg_c = int(reg_c.split(":")[1])
    
    f.readline()
    program = f.readline()[:-1]
    program = np.array(program.split(":")[-1].split(","), dtype=int)

In [26]:
run_program(program, reg_a, reg_b, reg_c)

'3,4,4,1,7,0,2,2'

In [27]:
def find_next(target, output):
    lo = int(target)
    hi = int(target + 8)

    for i in (range(lo,hi)):
        std_out = run_program(program[:-2], i, 0, 0)
        
        output[std_out] =  output.get(std_out, {})
        output[std_out][register_a] = output[std_out].get(register_a, [])
        output[std_out][register_a].append(i)
        
    return

In [29]:
start_time = time.time()

output = {}
possible_inputs = [0]

for instr in tqdm(np.flip(program)):
    
    for p in possible_inputs:
        find_next(int(p*8), output)
    
    new_possible_inputs = []
    for p in possible_inputs:
        new_possible_inputs = np.hstack((
            new_possible_inputs,
            (output[str(instr)].get(p, []))
        ))
    possible_inputs = new_possible_inputs
    print(f"{instr}\t{int(possible_inputs[0]):<20}{oct(int(possible_inputs[0])):<20}")
    
print(f"--- {1000*(time.time() - start_time):.1f} ms ---"  )

  0%|          | 0/16 [00:00<?, ?it/s]

0	7                   0o7                 
3	60                  0o74                
5	485                 0o745               
5	3884                0o7454              
4	31074               0o74542             
4	248594              0o745422            
4	1988754             0o7454222           
1	15910039            0o74542227          
3	127280319           0o745422277         
0	1018242552          0o7454222770        
5	8145940421          0o74542227705       
7	65167523371         0o745422277053      
3	521340186974        0o7454222770536     
1	4170721495792       0o74542227705360    
4	33365771966338      0o745422277053602   
2	266926175730705     0o7454222770536021  
--- 46.2 ms ---


In [86]:
output['0']

KeyError: '0'

{2: [20, 21], 3: [24, 28, 29], 5: [40]}

In [78]:
output = {}
possible_inputs = [0]
target = 4


for instr in tqdm(np.flip(program)):
    lo = int(target - min(64, target))
    hi = int(target + min(64, target))

    for i in (range(lo,hi)):
        std_out = run_program(program[:-2], i, 0, 0)
        
        output[std_out] =  output.get(std_out, {})
        output[std_out][register_a] = output[std_out].get(register_a, [])
        output[std_out][register_a].append(i)
        
    new_possible_inputs = []
    for p in possible_inputs:
        new_possible_inputs = np.hstack((
            new_possible_inputs,
            (output[str(instr)].get(p, []))
        ))
    possible_inputs = new_possible_inputs
    print(f"{instr}\t{int(possible_inputs[0]):<20}{oct(int(possible_inputs[0])):<20}")
    target = int(possible_inputs[0]) * 8
    
    

  0%|          | 0/16 [00:00<?, ?it/s]

0	4                   0o4                 
3	37                  0o45                
3	298                 0o452               
0	2390                0o4526              
5	19124               0o45264             
5	152996              0o452644            
3	1223973             0o4526445           
4	9791785             0o45264451          
5	78334283            0o452644513         
1	626674267           0o4526445133        
5	5013394138          0o45264451332       
7	40107153110         0o452644513326      
1	320857224887        0o4526445133267     
1	2566857799098       0o45264451332672    
4	20534862392784      0o452644513326720   
2	164278899142333     0o4526445133267275  
